Author: Omarov Ruslan  
Метод кратчайшего незамкнутого пути с использованием DSU

1. Найти пару точек $(i, j)$ с наименьшим $\rho_{i,j}$ и соединить их ребром;
2. Пока есть изолированные вершины продолжаем делать:  
    2.1 Найти изолированную точку, ближайшую к некоторой неизолированной;  
    2.2 Соединить эти две точки ребром;  
3. Удалить $K - 1$ самых длины ребер


In [134]:
# класс для задания системы непересекающихся множеств
class DSU:
    def __init__(self, n):
        self.size = [1] * (n + 1)
        self.parent = [i for i in range(n + 1)]

    def make_set(self, i):
        self.parent[i] = i

    def union_sets(self, a, b):
        a = self.find_set(a)
        b = self.find_set(b)
        if a != b:
            if self.size[a] < self.size[b]:
                a, b = b, a
            self.parent[b] = a
            self.size[a] += self.size[b]
            return True
        return False

    def find_set(self, v):
        if self.parent[v] == v:
            return v
        self.parent[v] = self.find_set(self.parent[v])
        return self.parent[v]

Будем решать задачу следующим образом  
1. Найдем минимальный остов в графе, при этом складывая ребра в какое-либо множество (например, макс-кучу, чтоб не нужно было лишний раз сортировать);
2. После того, как нашли остов, удаляем $K - 1$ ребер с максимальными весами из множества ребер, полученного на первом этапе (heappop $K - 1$ раз);
3. Построим график для визуализации полученного результата;

In [137]:
import heapq
from collections import defaultdict as dd
import pyvis as pv


class KNP:
    
    def __init__(self, n, k):
        self.edges = []
        self.n = n
        self.k = k
        self.dsu = DSU(n)
        self.ans = []
        self.clusters = dd(list)
        self.graph = dd(list)
        self.weights = {}
    
    def read(self, file_path):
        assert isinstance(file_path, str), f"Variable file_path shoulb be a str, not a {type(file_path)}"
        with open(file_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                i, j, w = tuple(map(int, line.split()))
                self.edges.append((w, i, j))
                self.weights[(i, j)] = w
                self.weights[(j, i)] = w
        self.edges.sort()
    
    def clusterize(self):
        counter = 0
        for w, i, j in self.edges:
            res = self.dsu.union_sets(i, j)
            if res:
                counter += 1
                heapq.heappush(self.ans, (-w, i, j))
            if counter == self.n - 1:
                break
        
        for i in range(self.k - 1):
            heapq.heappop(self.ans)   
    
    def get_graph(self, ls):
        ans = dd(list)
        for w, i, j in ls:
            ans[i].append(j)
            ans[j].append(i)
        return dict(ans)
    
    def export_graph(self, ls):
        g = self.get_graph(ls)
        labeled_nodes = {x: y for x, y in enumerate(list(range(1, self.n + 1)), 1)}
        visited = set()
        edges = []
        labeled = []
        for item in g:
            for neighbour in g[item]:
                if neighbour not in visited:
                    edges.append(f'{item} -- {neighbour} [ label = "{self.weights[(item, neighbour)] if (item, neighbour) in self.weights else self.weights[(neighbour, item)]}"]\n')
            visited.add(item)
            labeled.append(f'{item} [label="{labeled_nodes[item]}"]\n')

        edges = "".join(edges)
        labeled = "".join(labeled)
        return f"graph {{\n{labeled}{edges}}}"
    
    def show(self):
        nt = pv.network.Network("500px", "500px")
        f1 = open("test_full.dot", "w")
        f1.write(self.export_graph(self.edges))
        f1.close()
        nt.from_DOT("test_full.dot")
        nt.show("dot_full.html")
        f = open("test_res.dot", "w")
        f.write(self.export_graph(self.ans))
        f.close()
        nt.from_DOT("test_res.dot")
        nt.show("dot_res.html")
            
    def evaluate(self):
        self.reset()
        for w, i, j in self.ans:
            self.dsu.union_sets(i, j)
        
        for i in range(1, self.n + 1):
            self.clusters[self.dsu.parent[i]].append(i)
        return dict(self.clusters)
    
    def reset(self):
        self.dsu = DSU(self.n)
    
    

In [140]:
knp = KNP(11, 3)
knp.read("coordinates.txt")
knp.clusterize()
knp.show()